# 1) Import Package

In [ ]:
import time
import datetime
import random
import influxdb_client
import pandas as pd
from influxdb_client.client.write_api import SYNCHRONOUS

# 2) Import Data from previous

In [ ]:
def import_stock(stockname):
    name_temp = stockname.upper()
    dataframename = (f"{name_temp}")
    filename = (f"FULL_{name_temp}.csv")
    data = pd.read_csv(filename)
    data['time'] = pd.to_datetime(data['time'])
    data = data.sort_values(by=['time'],ascending=True)
    data.set_index('time')
    return data

In [ ]:
BA_full = import_stock("BA")
BWXT_full = import_stock("BWXT")
CACI_full = import_stock("CACI")
GD_full = import_stock("GD")
HII_full = import_stock("HII")
LDOS_full = import_stock("LDOS")
LHX_full = import_stock("LHX")
LMT_full = import_stock("LMT")
MANT_full = import_stock("MANT")
NOC_full = import_stock("NOC")
RTX_full = import_stock("RTX")
TXT_full = import_stock("TXT")

# 3) Set Start & End Time

In [ ]:
#Start Time

#%Y-%m-%d %H:%M:%S

#Earilest 2020-06-22 04:01:00

#1
#Select Start time between earliest timestamp or self input

#1.1 default
#starttime = LMT["time"].min()

#1.2 self setting
#Earilest 2020-06-22 04:01:00
starttime = datetime.datetime.strptime("2020-06-22 09:31:00", '%Y-%m-%d %H:%M:%S')

if starttime < datetime.datetime.strptime("2020-06-22 09:31:00", '%Y-%m-%d %H:%M:%S'):
    starttime = datetime.datetime.strptime("2020-06-22 09:31:00", '%Y-%m-%d %H:%M:%S')

#2
#current time counting
current_time = starttime

#3
#Select End time between earliest timestamp or self input

#Lasttime 2022-06-10 18:02:00
#3.1 default
#endtime = LMT["time"].max()

#3.2 self setting
#Earilest 2022-06-10 18:02:00
endtime = datetime.datetime.strptime("2022-06-10 18:02:00", '%Y-%m-%d %H:%M:%S')

if endtime > datetime.datetime.strptime("2022-06-10 18:02:00", '%Y-%m-%d %H:%M:%S'):
    endtime = datetime.datetime.strptime("2022-06-10 18:02:00", '%Y-%m-%d %H:%M:%S')


In [ ]:
#Select Only data after start time
BA = BA_full.loc[(BA_full['time'] >= starttime)]
BWXT = BWXT_full.loc[(BWXT_full['time'] >= starttime)]
CACI = CACI_full.loc[(CACI_full['time'] >= starttime)]
GD = GD_full.loc[(GD_full['time'] >= starttime)]
HII = HII_full.loc[(HII_full['time'] >= starttime)]
LDOS = LDOS_full.loc[(LDOS_full['time'] >= starttime)]
LHX = LHX_full.loc[(LHX_full['time'] >= starttime)]
LMT = LMT_full.loc[(LMT_full['time'] >= starttime)]
MANT = MANT_full.loc[(MANT_full['time'] >= starttime)]
NOC = NOC_full.loc[(NOC_full['time'] >= starttime)]
RTX = RTX_full.loc[(RTX_full['time'] >= starttime)]
TXT = TXT_full.loc[(TXT_full['time'] >= starttime)]

# 4) Update Stock at each time

In [ ]:
def stock_update(stockname):
    Now = stockname.loc[(stockname['time'] >= current_time)]
    This_Row = Now.loc[(Now['time'] == Now["time"].min())]
    
    
    #0 --> Name   
    show_stockname = This_Row["Stock_Name"].iloc[0]
    
    #1 --> Time
    show_time = This_Row["time"].iloc[0]
    #print(show_time)
    #print(type(show_time))
    
    #2 --> Price
    show_price = This_Row["open"].iloc[0]
    #print(show_price)
    #print(type(show_price))

    #3 --> Vol
    show_vol = This_Row["volume"].iloc[0]
    #print(show_vol)
    #print(type(show_vol))

    
    this_stock = [show_stockname,show_time,show_price,show_vol]
    #this_stock = [show_time,show_price,show_vol]
    
    return this_stock

# 5) Prepare stream source

In [ ]:
token = "_S-OBMurRIZfH_84JKd4PHPttIxv4nUJ4wOXSuY_dgIp5St1i8qHJDF9Jh48FupSXpUIjgTCJ7Yr_uNP6p-BGw=="
org = "KARN"
#org = 618198855986d409
bucket = "Watchlist"

# Store the URL of your InfluxDB instance
url="http://localhost:8086"

client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)
write_api = client.write_api(write_options=SYNCHRONOUS)

# 6) Stream into influxdb

In [ ]:
#time moving
time_change = datetime.timedelta(seconds=0)

while current_time <= endtime:
    
    #skip Saturday & Sunday
    if (current_time.weekday() == 5) or (current_time.weekday() == 6):
        time_change = datetime.timedelta(hours=24)
        current_time = current_time + time_change
        
    #skip stock market close
    elif current_time.hour >= 20:
        time_change = datetime.timedelta(hours=4) #If +8 time will skip to noon instantly must +4 to midnight then +4 to 04
        current_time = current_time + time_change
        time_change = datetime.timedelta(hours=4)
        current_time = current_time + time_change
        
    else:
        time_change = datetime.timedelta(minutes=1)
        
        current_status = list()
        
        BA_Now = stock_update(BA)
        BWXT_Now = stock_update(BWXT)
        CACI_Now = stock_update(CACI)
        GD_Now = stock_update(GD)
        HII_Now = stock_update(HII)
        LDOS_Now = stock_update(LDOS)
        LHX_Now = stock_update(LHX)
        LMT_Now = stock_update(LMT)
        MANT_Now = stock_update(MANT)
        NOC_Now = stock_update(NOC)
        RTX_Now = stock_update(RTX)
        TXT_Now = stock_update(TXT)
        
        current_status.append(BA_Now)
        current_status.append(BWXT_Now)
        current_status.append(CACI_Now)
        current_status.append(GD_Now)
        current_status.append(HII_Now)
        current_status.append(LDOS_Now)
        current_status.append(LHX_Now)
        current_status.append(LMT_Now)
        current_status.append(MANT_Now)
        current_status.append(NOC_Now)
        current_status.append(RTX_Now)
        current_status.append(TXT_Now)
        
        current_time = current_time + time_change
            
        BA_write = influxdb_client.Point("Stock").tag("StockName",  
                "Boeing_BA").field("Price", (BA_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=BA_write) 
        
        BWXT_write = influxdb_client.Point("Stock").tag("StockName",  
                "BWX-NukeTech_BWXT").field("Price", (BWXT_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=BWXT_write)
        
        CACI_write = influxdb_client.Point("Stock").tag("StockName",  
                "CA-Analysis_CACI").field("Price", (CACI_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=CACI_write)
        
        GD_write = influxdb_client.Point("Stock").tag("StockName",  
                "Gen.-Dynamic_GD").field("Price", (GD_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=GD_write)
        
        HII_write = influxdb_client.Point("Stock").tag("StockName",  
                "HII-Shipyard_HII").field("Price", (HII_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=HII_write) 
        
        LDOS_write = influxdb_client.Point("Stock").tag("StockName",  
                "ItHolding_LDOS").field("Price", (LDOS_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=LDOS_write)
        
        LHX_write = influxdb_client.Point("Stock").tag("StockName",  
                "L3Harris_LHX").field("Price", (LHX_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=LHX_write)
        
        LMT_write = influxdb_client.Point("Stock").tag("StockName",  
                "Lockheed_LMT").field("Price", (LMT_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=LMT_write)    
                
        MANT_write = influxdb_client.Point("Stock").tag("StockName",  
                "Mantech_MANT").field("Price", (MANT_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=MANT_write)
        
        NOC_write = influxdb_client.Point("Stock").tag("StockName",  
                "NorthropG_NOC").field("Price", (NOC_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=NOC_write)
        
        RTX_write = influxdb_client.Point("Stock").tag("StockName",  
                "Raytheon_RTX").field("Price", (RTX_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=RTX_write)
        
        TXT_write = influxdb_client.Point("Stock").tag("StockName",  
                "Textron_TXT").field("Price", (TXT_Now[2]).round(3))
        write_api.write(bucket=bucket, org=org, record=TXT_write)

        print(f"Boeing_BA --> Time : {BA_Now[1]} Price : {(BA_Now[2]).round(3)} | Vol : {BA_Now[3]}")
        print(f"BWX-NukeTech_BWXT --> Time : {BWXT_Now[1]} Price : {(BWXT_Now[2]).round(3)} | Vol : {BWXT_Now[3]}")
        print(f"CA-Analysis_CACI --> Time : {CACI_Now[1]} Price : {(CACI_Now[2]).round(3)} | Vol : {CACI_Now[3]}")
        print(f"Gen.-Dynamic_GD --> Time : {GD_Now[1]} Price : {(GD_Now[2]).round(3)} | Vol : {GD_Now[3]}")
        print(f"HII-Shipyard_HII --> Time : {HII_Now[1]} Price : {(HII_Now[2]).round(3)} | Vol : {HII_Now[3]}")
        print(f"ItHolding_LDOS --> Time : {LDOS_Now[1]} Price : {(LDOS_Now[2]).round(3)} | Vol : {LDOS_Now[3]}")
        print(f"L3Harris_LHX --> Time : {LHX_Now[1]} Price : {(LHX_Now[2]).round(3)} | Vol : {LHX_Now[3]}")
        print(f"Lockheed_LMT --> Time : {LMT_Now[1]} Price : {(LMT_Now[2]).round(3)} | Vol : {LMT_Now[3]}")
        print(f"Mantech_MANT --> Time : {MANT_Now[1]} Price : {(MANT_Now[2]).round(3)} | Vol : {MANT_Now[3]}")
        print(f"NorthropG_NOC --> Time : {NOC_Now[1]} Price : {(NOC_Now[2]).round(3)} | Vol : {NOC_Now[3]}")
        print(f"Raytheon_RTX --> Time : {RTX_Now[1]} Price : {(RTX_Now[2]).round(3)} | Vol : {RTX_Now[3]}")
        print(f"Textron_TXT --> Time : {TXT_Now[1]} Price : {(TXT_Now[2]).round(3)} | Vol : {TXT_Now[3]}")


        #p = influxdb_client.Point(my_measurement).tag(location,  
        #Prague).field(temperature, float(randomlist[0]))
        
        #for each_now in current_status:
            
                #BA = influxdb_client.Point("Watchlist").tag("Boeing_BA",  
                #"Defense").field("Price", float(randomlist[0]))
    
                #write_api.write(bucket=bucket, org=org, record=BA)
        
                #print(f"Stock : {each_now[0]} | Price : {(each_now[2]).round(3)} | Vol : {each_now[3]}")